<a href="https://colab.research.google.com/github/RyanPerrina/wine_classification/blob/main/wine_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


# Descrizione del problema
Il dataset in questione riguarda le varianti di vino rosso portoghese "Vinho Verde". Il dataset descrive la qualità del vino in relazione ai componenti chimici presenti. 

In [ ]:
import os.path
if not os.path.exists("dataset-wine.csv"):
	from urllib.request import urlretrieve
	urlretrieve("https://www.kaggle.com/datasets/yasserh/wine-quality-dataset/download","dataset-wine.csv")
data = pd.read_csv("dataset-wine.csv")